# Preprocessing

This notebook is needed for preparing the input datasets: upper case and etc.

### The input datasets should be in the following format:
- Company name (string)
- Marketplace (string)
- Country (string)
- State (string)
- City (string)
- Zip Code (string)
- Street (string)
- URL (string)
- Industry (SIC)

In [1]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import sys
import numpy as np
import string
import collections
from textblob import TextBlob
import Levenshtein
import binascii

import matplotlib.pyplot as plt
import random
import time

In [2]:
def df_import():
    # import FR data (adjustments for delimeters and encoding - latin)
    wordbook_name = "~/Dropbox/Botva/TUM/Master_Thesis/datasets/processed_files/france_rna_processed.csv"
    df = pd.read_csv(wordbook_name, encoding='latin-1', sep = ';', error_bad_lines=False)
#    df = pd.read_csv("~/Dropbox/Botva/TUM/Master_Thesis/object-identification/datasets/raw_files/rna_waldec_20201201_dpt_01.csv", error_bad_lines=False)
#    df = df.astype(str)
    print(df)
    print(df.dtypes)
    return df

In [3]:
def df_prepare(df):
    df = df.apply(lambda x: x.astype(str).str.upper())
    df['name'] = df['name'].apply(lambda x: x.replace('.',''))
    df['name'] = df['name'].str.replace('[^0-9a-zA-Z]+', ' ')
    df['name'] = df['name'].str.replace(' +', ' ')
    df['name_split'] = df['name'].str.split(' ')
    print(df)
    print(df.dtypes)

    return df

In [4]:
def frequent_words(df_processed):
    b = TextBlob("bonjour")
    b.detect_language()
    print(df_processed['name'])
    all_words = df_processed['name']    
    all_words_cleaned = []

    for text in all_words:
        text = [x.strip(string.punctuation) for x in text]
        all_words_cleaned.append(text)

    all_words_cleaned[0]

    text_words = [" ".join(text) for text in all_words_cleaned]
    final_text_words = " ".join(text_words)
    #final_text_words[:1000]

    print(all_words)
    stopwords = set(STOPWORDS)
    stopwords.update(["LE","DE","LA","ET","DES","DU","LES","EN","ET","A","POUR","SUR","SOU","S","D","L"])

    wordcloud_names = WordCloud(stopwords=stopwords, background_color="white", max_font_size=50, max_words=100).generate(final_text_words)

    # Lines 4 to 7
    plt.figure(figsize = (15,15))
    plt.imshow(wordcloud_names, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    filtered_words = [word for word in final_text_words.split() if word not in stopwords]
    counted_words = collections.Counter(filtered_words)

    word_count = {}

    for letter, count in counted_words.most_common(100):
        word_count[letter] = count

    for i,j in word_count.items():
        print('Word: {0}, count: {1}'.format(i,j))
    
    return word_count

Levenstein Distance

In [ ]:
def levenstein_distance(texts_1,texts_2):
    lvn_array = np.zeros((len(texts_1),len(texts_2)))
    print(lvn_array)

    text_1_num = 0
    text_2_num = 0

    for text_1 in texts_1:
        for text_2 in texts_2:
            lvn_array[text_1_num,text_2_num] = Levenshtein.ratio(text_1,text_2)
            text_2_num += 1 
        text_2_num = 0    
        text_1_num += 1 
    return lvn_array

In [5]:
df = df_import()

         local_id                                               name country  \
0      W011000001                                         VENT D'EST  FRANCE   
1      W011000002                                UNE NOTE DE PARTAGE  FRANCE   
2      W011000003                                         CHANDELAIN  FRANCE   
3      W011000004                                   LEYMENT-MATERIEL  FRANCE   
4      W011000005                          ASSOCIATION 'TOILES EMOI'  FRANCE   
...           ...                                                ...     ...   
18119  W014004922                                  AU BONHEUR D'ELDA  FRANCE   
18120  W014004923                                 JE S'AIME ET VOUS?  FRANCE   
18121  W014004924  'ASSOCIATION COMMUNALE DE CHASSE AGREE DE MART...  FRANCE   
18122  W014004925  'ASSOCIATION COMMUNALE DE CHASSE AGREEE DE GRO...  FRANCE   
18123  W014004926                                    L'ATTITUDE NORD  FRANCE   

       state               city    zip 

In [6]:
df_processed = df_prepare(df)

         local_id                                               name country  \
0      W011000001                                         VENT D EST  FRANCE   
1      W011000002                                UNE NOTE DE PARTAGE  FRANCE   
2      W011000003                                         CHANDELAIN  FRANCE   
3      W011000004                                   LEYMENT MATERIEL  FRANCE   
4      W011000005                           ASSOCIATION TOILES EMOI   FRANCE   
...           ...                                                ...     ...   
18119  W014004922                                  AU BONHEUR D ELDA  FRANCE   
18120  W014004923                                 JE S AIME ET VOUS   FRANCE   
18121  W014004924   ASSOCIATION COMMUNALE DE CHASSE AGREE DE MART...  FRANCE   
18122  W014004925   ASSOCIATION COMMUNALE DE CHASSE AGREEE DE GRO...  FRANCE   
18123  W014004926                                    L ATTITUDE NORD  FRANCE   

      state               city    zip  

In [8]:
#creating shingles_dict
def create_shingles_dict(texts,k):
    shingles_list = set()

    for text in texts:
    #    print(text)
    #    text = text.encode()
        if len(text) >= k:
            shingles = [text[i:i + k] for i in range(len(text) - k + 1)]
        else:
            shingles = [text + ' ' * (k - len(text))]
        for shingle in shingles:
            if shingle not in shingles_list: #check, maybe if is not needed bc its a set
                shingles_list.add(shingle)

    shingles_list = sorted(shingles_list)
    shingles_dict = dict(zip(shingles_list,range(len(shingles_list))))
    return shingles_list, shingles_dict

k = 3
texts = df_processed['name']
shingles_list, shingles_dict = create_shingles_dict(texts,k)


In [9]:
#converting docs to shingles
def create_doc_shingles(texts,k):
    docs = [[] for i in range(len(texts))]

    for doc, text in zip(docs, texts):
        if len(text) >= k:
            shingles = [text[i:i + k] for i in range(len(text) - k + 1)]
        else:
            shingles = [text + ' ' * (k - len(text))]
        for shingle in shingles:
            doc.append(shingles_dict[shingle])
    return docs

docs = create_doc_shingles(texts,k)

In [10]:
#creating signatures array
def create_signatures_array(docs,shingles_list,signature_size):
    signatures = np.zeros((signature_size, len(docs))) #create an array

    shingles_shuffled = [i for i in range(len(shingles_list))]

    for signature in signatures:
        random.shuffle(shingles_shuffled)    
        for doc_index, doc in enumerate(docs):
            doc_a = [shingles_shuffled[i] for i in doc]
            signature[doc_index] = min(doc_a)
    return signatures

signature_size = 50
signatures = create_signatures_array(docs,shingles_list,signature_size)

In [11]:
def create_buckets(signatures,bands_number):
    r = int(len(signatures)/bands_number) #rows per band

    buckets_bands = [{} for i in range(bands_number)]

    for band, buckets in zip(range(bands_number),buckets_bands):
        for i in range(len(signatures[0])):
            buckets.setdefault(tuple(signatures[band*r:band*r+r,i]),[]).append(i)
        filtered = {key:item for key, item in buckets.items() if len(item)>1}
        buckets.clear()
        buckets.update(filtered)
    return buckets_bands

bands_number = 5
buckets_bands = create_buckets(signatures,bands_number)

In [12]:
def jaccard(list1, list2):
    intersection = len(set(list1).intersection(list2))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return intersection/union

def create_matches(buckets_bands):
    matches = {}
    for buckets in buckets_bands:
        for key, values_list in buckets.items():
            for value_1 in values_list:
                for value_2 in values_list:
                    if value_2 > value_1 and (value_1,value_2) not in matches: 
                        matches.setdefault((value_1,value_2),[]).append(jaccard(docs[value_1], docs[value_2]))
                        #print(value_1)
                        #print(value_2)
    #                    print(texts[value_1])
    #                    print(texts[value_2])
    #                    print(jaccard(docs[value_1], docs[value_2]))
    return matches

#threshold = 0.7
matches = create_matches(buckets_bands)
matches


{(0, 10173): [1.0],
 (18, 456): [1.0],
 (18, 469): [1.0],
 (18, 550): [1.0],
 (18, 569): [1.0],
 (18, 600): [1.0],
 (18, 3185): [1.0],
 (18, 4683): [1.0],
 (18, 4736): [1.0],
 (18, 16348): [1.0],
 (18, 16356): [0.6785714285714286],
 (456, 469): [1.0],
 (456, 550): [1.0],
 (456, 569): [1.0],
 (456, 600): [1.0],
 (456, 3185): [1.0],
 (456, 4683): [1.0],
 (456, 4736): [1.0],
 (456, 16348): [1.0],
 (456, 16356): [0.6785714285714286],
 (469, 550): [1.0],
 (469, 569): [1.0],
 (469, 600): [1.0],
 (469, 3185): [1.0],
 (469, 4683): [1.0],
 (469, 4736): [1.0],
 (469, 16348): [1.0],
 (469, 16356): [0.6785714285714286],
 (550, 569): [1.0],
 (550, 600): [1.0],
 (550, 3185): [1.0],
 (550, 4683): [1.0],
 (550, 4736): [1.0],
 (550, 16348): [1.0],
 (550, 16356): [0.6785714285714286],
 (569, 600): [1.0],
 (569, 3185): [1.0],
 (569, 4683): [1.0],
 (569, 4736): [1.0],
 (569, 16348): [1.0],
 (569, 16356): [0.6785714285714286],
 (600, 3185): [1.0],
 (600, 4683): [1.0],
 (600, 4736): [1.0],
 (600, 16348): [1

In [13]:
#generate df with all potential matches 
def create_df_with_attributes(matches,texts):
    df_matches = pd.DataFrame.from_dict(matches, orient='index')
    df_matches['matches_tuple'] = df_matches.index
    df_matches[['doc_1','doc_2']] = pd.DataFrame(df_matches['matches_tuple'].tolist(), index=df_matches.index) 
    df_matches = df_matches.drop(['matches_tuple'], axis=1)
    df_matches = df_matches.reset_index(drop=True)

    df_texts = pd.DataFrame(texts) 
    df_texts['index'] = df_texts.index

    df_matches_full = pd.merge(df_matches, df_texts,  how='left', left_on=['doc_1'], right_on = ['index'])
    df_matches_full = df_matches_full.drop(['index'], axis=1)
    df_matches_full = pd.merge(df_matches_full, df_texts,  how='left', left_on=['doc_2'], right_on = ['index'])
    df_matches_full = df_matches_full.drop(['index'], axis=1)
    return df_matches_full

df_matches_full = create_df_with_attributes(matches,texts)
df_matches_full


,0,doc_1,doc_2,name_x,name_y
0,1.000000,0,10173,VENT D EST,VENT D EST
1,1.000000,18,456,SOU DES ECOLES LAIQUES,SOU DES ECOLES LAIQUES
2,1.000000,18,469,SOU DES ECOLES LAIQUES,SOU DES ECOLES LAIQUES
3,1.000000,18,550,SOU DES ECOLES LAIQUES,SOU DES ECOLES LAIQUES
4,1.000000,18,569,SOU DES ECOLES LAIQUES,SOU DES ECOLES LAIQUES
...,...,...,...,...,...
11353,0.702703,16945,17593,ECOLE SANS FRONTIERE PROMO 2014,ECOLE SANS FRONTIERE PROMOTION 2019
11354,0.722222,17149,17473,ECOLE SANS FRONTIERE PROMOTION 2015,ECOLE SANS FRONTIERE PROMO 2018
11355,0.722222,17342,17473,ECOLE SANS FRONTIERE PROMOTION 2017,ECOLE SANS FRONTIERE PROMO 2018
11356,0.702703,17473,17593,ECOLE SANS FRONTIERE PROMO 2018,ECOLE SANS FRONTIERE PROMOTION 2019


In [ ]:


print(signatures[:,0])
print(jaccard(docs[0], docs[0]))

In [ ]:

def jaccard(list1, list2):
#    print(len(list(set(list1).intersection(list2))))
    intersection = len(set(list1).intersection(list2))
#    print(len(list1))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return intersection/union

print(signatures[:,0])
print(jaccard(docs[0], docs[0]))




threshold = 0.5
a = len(signatures[0])
bucket = {}

for band in range(bands):
    for i in range(len(signatures[0])):
    #    t0 = time.time()
        for j in range(len(signatures[0])):
            if i<j:
                if jaccard(signatures[band*r:band*r+r,i], signatures[band*r:band*r+r,j])>threshold:
#                    print(jaccard(signatures[band*r:band*r+r,i], signatures[band*r:band*r+r,j]))
#                    print(i,j)
                    print(texts[i])
                    print(texts[j])
                    bucket.setdefault(i,[]).append(j)
                    print(bucket)
    #    elapsed = (time.time() - t0)            
    #    print("\nChecking one doc to all others took %.2fsec" % elapsed)  
    #    b = (a-i)*elapsed/60/60
    #    print("\nExpected left time: %.2fhours" % b)  
 


#compare row by row

In [ ]:
min(len(d) for d in texts)

In [ ]:
Levenshtein.ratio(df_processed['name'],df_processed['name'])


In [ ]:
from textdistance import levenshtein

df_1 = df_processed['name']
df_2 = df_processed['name']
df_1.apply(lambda x: levenshtein.distance(df_1['name'], df_2['name']), axis=1)

In [ ]:
!{sys.executable} -m pip install textdistance